In [1]:
import xarray as xr
import fsspec
import pandas as pd
from seapopym.standard.units import StandardUnitsLabels
import hvplot.xarray

In [2]:
station = "Hot"

## Load DATA


In [3]:
data_obs = xr.open_dataset(f"../data/1_products/{station}_obs_zoo_climato_monthly_2002_2015.zarr", engine="zarr").load()
data_obs

<xarray.Dataset> Size: 4kB
Dimensions:    (time: 156, latitude: 1, longitude: 1, layer: 1)
Coordinates:
  * latitude   (latitude) float64 8B 22.75
  * layer      (layer) int64 8B 0
  * longitude  (longitude) int64 8B -158
  * time       (time) datetime64[ns] 1kB 2002-01-31 2002-02-28 ... 2014-12-31
Data variables:
    day        (time, latitude, longitude, layer) float64 1kB 1.369 ... 1.675
    night      (time, latitude, longitude, layer) float64 1kB 2.055 ... 2.569
Attributes:
    desc:     Aggregated and smoothed observations using lowess filtre.

In [4]:
data_cmems = xr.open_dataset(f"../data/1_products/{station}_cmems.zarr", engine="zarr").load()
first_year = str(data_obs.indexes["time"].to_pydatetime()[0].year)
last_year = str(data_obs.indexes["time"].to_pydatetime()[-1].year)
data_cmems = data_cmems.sel(time=slice(first_year, last_year))
data_cmems

<xarray.Dataset> Size: 836kB
Dimensions:              (latitude: 1, longitude: 1, time: 4748, depth: 3)
Coordinates:
  * depth                (depth) int64 24B 0 1 2
  * latitude             (latitude) float64 8B 22.75
  * longitude            (longitude) int64 8B -158
  * time                 (time) datetime64[ns] 38kB 2002-01-01 ... 2014-12-31
Data variables: (12/13)
    T                    (latitude, longitude, time, depth) float64 114kB 23....
    U                    (latitude, longitude, time, depth) float64 114kB -0....
    V                    (latitude, longitude, time, depth) float64 114kB 0.0...
    mnkc_epi             (latitude, longitude, time) float64 38kB 0.2207 ... ...
    mnkc_hmlmeso         (latitude, longitude, time) float64 38kB 0.5442 ... ...
    mnkc_lmeso           (latitude, longitude, time) float64 38kB 1.842 ... 1...
    ...                   ...
    mnkc_mumeso          (latitude, longitude, time) float64 38kB 0.4258 ... ...
    mnkc_umeso           (latitude, longitude, time) float64 38kB 0.3893 ... ...
    npp                  (latitude, longitude, time) float64 38kB 321.6 ... 3...
    pelagic_layer_depth  (latitude, longitude, time, depth) float64 114kB 113...
    zeu                  (latitude, longitude, time) float64 38kB 75.7 ... 70.6
    zooc                 (latitude, longitude, time) float64 38kB 0.2347 ... ...
Attributes:
    Conventions:  CF-1.7
    history:      Created on 2024-10-17
    institution:  CLS
    references:   http://www.cls.fr; http://www.seapodym.eu
    source:       SEAPODYM-LMTL 3.0.0
    title:        Global ocean low and mid trophic levels biomass content hin...

## Climato


In [5]:
for i in range(1, 13):
    print(
        f"Month {i} : ",
        set(
            pd.DataFrame({"month": data_cmems.indexes["time"].month, "day": data_cmems.indexes["time"].day})
            .query(f"month == {i}")["day"]
            .unique()
        ),
    )

Month 1 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}
Month 2 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29}
Month 3 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}
Month 4 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30}
Month 5 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}
Month 6 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30}
Month 7 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}
Month 8 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}


In [6]:
data_cmems_climato = data_cmems.groupby("time.dayofyear").mean()
data_cmems_climato

<xarray.Dataset> Size: 64kB
Dimensions:              (dayofyear: 366, latitude: 1, longitude: 1, depth: 3)
Coordinates:
  * depth                (depth) int64 24B 0 1 2
  * latitude             (latitude) float64 8B 22.75
  * longitude            (longitude) int64 8B -158
  * dayofyear            (dayofyear) int64 3kB 1 2 3 4 5 ... 362 363 364 365 366
Data variables: (12/13)
    T                    (dayofyear, latitude, longitude, depth) float64 9kB ...
    U                    (dayofyear, latitude, longitude, depth) float64 9kB ...
    V                    (dayofyear, latitude, longitude, depth) float64 9kB ...
    mnkc_epi             (dayofyear, latitude, longitude) float64 3kB 0.2813 ...
    mnkc_hmlmeso         (dayofyear, latitude, longitude) float64 3kB 0.8012 ...
    mnkc_lmeso           (dayofyear, latitude, longitude) float64 3kB 2.116 ....
    ...                   ...
    mnkc_mumeso          (dayofyear, latitude, longitude) float64 3kB 0.5679 ...
    mnkc_umeso           (dayofyear, latitude, longitude) float64 3kB 0.4899 ...
    npp                  (dayofyear, latitude, longitude) float64 3kB 302.2 ....
    pelagic_layer_depth  (dayofyear, latitude, longitude, depth) float64 9kB ...
    zeu                  (dayofyear, latitude, longitude) float64 3kB 76.5 .....
    zooc                 (dayofyear, latitude, longitude) float64 3kB 0.261 ....
Attributes:
    Conventions:  CF-1.7
    history:      Created on 2024-10-17
    institution:  CLS
    references:   http://www.cls.fr; http://www.seapodym.eu
    source:       SEAPODYM-LMTL 3.0.0
    title:        Global ocean low and mid trophic levels biomass content hin...

## Expand to original time series interval


In [7]:
all_years = []
for year in range(int(data_cmems.time.min().dt.year), int(data_cmems.time.max().dt.year + 1)):
    slice_max = 366
    if not pd.Timestamp(f"{year}-12-31").is_leap_year:
        slice_max = 365

    yearly_data = data_cmems_climato.sel(dayofyear=slice(1, slice_max)).assign_coords(
        time=(
            "dayofyear",
            pd.date_range(
                start=f"{year}-01-01",
                end=f"{year}-12-31",
                freq="D",
                name="time",
            ),
        ),
    )

    all_years.append(yearly_data.swap_dims({"dayofyear": "time"}))
data_cmems_climato_expanded = xr.concat(all_years, dim="time").drop_vars("dayofyear")
data_cmems_climato_expanded.time.attrs = data_cmems.time.attrs
data_cmems_climato_expanded

<xarray.Dataset> Size: 836kB
Dimensions:              (time: 4748, latitude: 1, longitude: 1, depth: 3)
Coordinates:
  * depth                (depth) int64 24B 0 1 2
  * latitude             (latitude) float64 8B 22.75
  * longitude            (longitude) int64 8B -158
  * time                 (time) datetime64[ns] 38kB 2002-01-01 ... 2014-12-31
Data variables: (12/13)
    T                    (time, latitude, longitude, depth) float64 114kB 23....
    U                    (time, latitude, longitude, depth) float64 114kB -0....
    V                    (time, latitude, longitude, depth) float64 114kB -0....
    mnkc_epi             (time, latitude, longitude) float64 38kB 0.2813 ... ...
    mnkc_hmlmeso         (time, latitude, longitude) float64 38kB 0.8012 ... ...
    mnkc_lmeso           (time, latitude, longitude) float64 38kB 2.116 ... 2...
    ...                   ...
    mnkc_mumeso          (time, latitude, longitude) float64 38kB 0.5679 ... ...
    mnkc_umeso           (time, latitude, longitude) float64 38kB 0.4899 ... ...
    npp                  (time, latitude, longitude) float64 38kB 302.2 ... 3...
    pelagic_layer_depth  (time, latitude, longitude, depth) float64 114kB 114...
    zeu                  (time, latitude, longitude) float64 38kB 76.5 ... 75.31
    zooc                 (time, latitude, longitude) float64 38kB 0.261 ... 0...
Attributes:
    Conventions:  CF-1.7
    history:      Created on 2024-10-17
    institution:  CLS
    references:   http://www.cls.fr; http://www.seapodym.eu
    source:       SEAPODYM-LMTL 3.0.0
    title:        Global ocean low and mid trophic levels biomass content hin...

## Plot


In [8]:
data_cmems_climato_expanded.mean(["latitude", "longitude"]).hvplot.line(
    x="time",
    groupby="depth",
    title=f"Climato {station} - CMEMS T",
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'094eaefa-dff9-43ca-88f2-bf1e182bbbad': {'version…

## Export


In [9]:
data_cmems_climato_expanded.to_zarr(
    f"../data/1_products/{station}_cmems_climato.zarr",
    mode="w",
)